In [1]:
path_file = "../data/COLIEE2024statute_data-English/train/"

In [1]:
from os import listdir
from os.path import isfile, join

def get_all_files_from_path(mypath):
    filenames = [join(mypath, f) for f in listdir(mypath) if isfile(join(mypath, f))]
    return filenames

from bs4 import BeautifulSoup
import re
import json

def get_article(articles):
    result = {}
    current_statue = "(non-statute)"
    for i in re.split(r"(.*)", articles.strip()):
        if len(i) == 0 or i == "\n":
            continue
        if re.search(r"^\(.*\)$", i):
            current_statue = i.strip()
            if current_statue not in result:
                result.update({current_statue: []})
        else:
            if current_statue not in result:
                result.update({current_statue: []})
            result[current_statue].append(i)
    return result

def build_test(filename):
    result = {}
    with open(filename, 'r') as f:
        data = f.read()

    data = BeautifulSoup(data, "xml").find_all('pair')
    for i in data:
        id = i.get('id')
        result.update({id: {}})
        result[id].update({"label": i.get('label')})
        articles = i.find('t1').text.strip()
        # articles = get_article(articles)
        result[id].update({"result": articles})
        result[id].update({"content": i.find('t2').text.strip()})
    return result

def write_json(filename, data):
    with open(filename, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)

import xml.etree.ElementTree as Et
import glob

def format_first_line(text):
    lines = text.split("\n")
    results = []
    for line in lines:
        if line[0] == "":
            continue
        if line[0] == "(" and line[-1] == ")":
            continue
        results.append(line)
    return "\n".join(results)

def load_samples(filexml):
    # try:
        tree = Et.parse(filexml)
        root = tree.getroot()
        samples = []
        for i in range(0, len(root)):
            sample = {'result': []}
            for j, e in enumerate(root[i]):
                if e.tag == "t1":
                    sample['result'] = format_first_line(e.text.strip())
                elif e.tag == "t2":
                    question = e.text.strip()
                    sample['content'] = question if len(question) > 0 else None
            sample.update(
                {'index': root[i].attrib['id'], 'label': root[i].attrib.get('label', "N")})
            # filter the noise samples
            if sample['content'] is not None:
                samples.append(sample)
            else:
                print("[Important warning] samples {} is ignored".format(sample))
        return samples

def load_test_data_samples(path_folder_base, test_id):
    data = []
    test = load_samples(f"{path_folder_base}/riteval_{test_id}.xml")
    for file_path in glob.glob(f"{path_folder_base}/riteval_{test_id}.xml"):
        data = data + load_samples(file_path)
    return data


def load_all_data_samples(path_folder_base):
    data = []
    for file_path in glob.glob("{}/*.xml".format(path_folder_base)):
        data = data + load_samples(file_path)
    return data

def check_false_labels(pred, false_labels):
	for label in false_labels:
		if label in pred:
			return True
	return False

import json
def readfile(filename):
    f = open(filename)
    data = json.load(f)
    return data

def writefile(data, filename):
    # Serializing json
    json_object = json.dumps(data, indent=1)
    # Writing to sample.json
    with open(filename, "w") as outfile:
        outfile.write(json_object)

def load_jsonl(file):
    with open(file, encoding="utf-8") as f:
        data = [json.loads(line) for line in f]
    return data

In [2]:
from transformers import AutoTokenizer, BloomForCausalLM, AutoTokenizer, AutoModelForSeq2SeqLM
import torch

model_name = "google/flan-t5-xxl"
cache_dir = "/home/congnguyen/drive/.cache"
tokenizer = AutoTokenizer.from_pretrained(model_name, cache_dir=cache_dir)
model = AutoModelForSeq2SeqLM.from_pretrained(
		model_name, device_map="auto", cache_dir=cache_dir, torch_dtype=torch.float16, load_in_8bit=True
	)
# model = model.cuda()

/home/congnguyen/drive/miniconda3/envs/myenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:09<00:00,  1.92s/it]


In [3]:
from tqdm import tqdm

def format_output(text):
	CLEANR = re.compile('<.*?>') 
	cleantext = re.sub(CLEANR, '', text)
	return cleantext.strip().lower()

def prompting(premise, hypothesis, template=None):
    text = template.replace("{{premise}}", premise).replace("{{hypothesis}}", hypothesis)
    return text

In [43]:
test_file = "../data/COLIEE2024statute_data-English/fewshot.json/rewrite/riteval_R05_en.jsonl"
data = load_jsonl(test_file)
data

[{'result': 'Article 537 (1) If one of the parties promises in a contract to render a certain performance to a third party, the third party has the right to claim that performance directly from the obligor.\n(2) The validity of the contract referred to in the preceding paragraph is not impaired even if a third party does not exist or a third party is not specified at the time of its formation.\n(3) In the case referred to in paragraph (1), rights of the third party accrue when the third party has manifested intention of availing of the benefit of the contract under that paragraph to the obligor.',
  'content': 'The validity of a third party beneficiary contract made by A with C, the mother of B, to give land X owned by A to B gratuitously, is not impaired even if B is an unborn child as of the time of the formation of the contract.',
  'index': 'R05-01-A',
  'label': 'Y',
  'rewrite': 'Article 537\nIf one of the parties promises in a contract to render a certain performance to a third 

In [44]:
template_prompt = "Document:{{premise}}\nQuestion: {{hypothesis}}? No or Yes"
# template_prompt = "{{premise}}\nQuestion: {{hypothesis}}? "

In [45]:
def write_test(data, output_file):
    f = open(output_file, "w", encoding="utf-8")
    for k in data:
        f.write(k.strip()+" "+data[k]+" "+"CAPTAIN\n")

In [46]:
import os
count = 0
count2 = 0
result = {}
# output_folder = "../output/R05.task4.CAPTAIN3_riteval_R05_en.xml"
for item in tqdm(data):
    label = item["label"]
    hypothesis = item["content"].strip()
    # premise = item["result"].strip()
    if "rewrite" in test_file:
        premise = item["rewrite"].strip()
    else: premise = item["result"].strip()
    #Important: You must use dot-product, not cosine_similarity
    text = prompting(premise, hypothesis, template_prompt)
    inputs = tokenizer(text, return_tensors="pt")["input_ids"].cuda()
    outputs = model.generate(inputs, max_new_tokens=10)
    output_text = format_output(tokenizer.decode(outputs[0]).replace(text, "").split("\n")[-1])
    
    if "yes" in output_text or "true" in output_text:
        output_text = "Y"
    else:
        output_text = "N"
        
    if output_text == label:
        count += 1
    else:
        print(item["index"], ": ", output_text, " - ", label)
        print(text)
    # result.update({item["index"]: output_text})
    # write_test(result, output_folder)
    ####################################
    # result = split_hypo(hypothesis)
    # if result == None: 
    #     if output_text == label:
    #         count2 += 1
    #     continue
        
    # check = []
    # text_check = []
    # for t in result:
    #     text = prompting(premise, t.strip(), template_prompt)
    #     inputs = tokenizer(text, return_tensors="pt")["input_ids"].cuda()
    #     outputs = model.generate(inputs, max_new_tokens=10)
    #     output_text = format_output(tokenizer.decode(outputs[0]).replace(text, "").split("\n")[-1])
    #     # print(item["index"], ": ", output_text, " - ", label)
    #     check.append(output_text)
    #     text_check.append(t.strip())
    # # print(check)
    # # print(text_check)
    # flag = check[0]
    # for k in check:
    #     if flag!=k:
    #         flag = "false"
    # if "yes" in flag or "true" in flag:
    #     output_text = "Y"
    # else:
    #     output_text = "N"
    # if output_text == label:
    #     count2 += 1
    # print(item["index"], ": ", output_text, " - ", label)
    # print("=========================")


  2%|██▊                                                                                                                                                        | 2/109 [00:00<00:38,  2.78it/s]

R05-02-A :  Y  -  N
Document:Article 712
If a minor has inflicted damage on another person but did not have sufficient intellectual capacity to appreciate their own liability for that act. that minor is not liable to compensate for that act.
Question: Mental capacity means the capacity to appreciate one's own liability for one's acts.? No or Yes


  3%|████▎                                                                                                                                                      | 3/109 [00:01<00:36,  2.92it/s]

R05-02-I :  N  -  Y
Document:Article 3-2
If the person making a juridical act did not have mental capacity when manifesting the relevant intention. the juridical act is void.
Question: If a party to a contract did not have mental capacity when manifesting the relevant intention, the nullity of the contract may be asserted even against a third party in good faith and acting without negligence.? No or Yes


  9%|██████████████▏                                                                                                                                           | 10/109 [00:03<00:33,  2.93it/s]

...

 97%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊    | 106/109 [00:38<00:01,  2.70it/s]

R05-36-I :  N  -  Y
Document:Article 466-4
The provisions of Article 466, paragraph (3) do not apply to an attaching obligee that has enforced compulsory execution against a claim that is subject to the manifestation of intention to restrict assignment. The provisions of Article 466, paragraph (3) do not apply to the attaching obligee. The claim that the attaching obligee has enforced compulsory execution against is subject to the manifestation of intention to restrict assignment.
if a third party such as the assignee knew or did not know due to gross negligence that the manifestation of intention to restrict assignment was made the obligor may refuse to perform the obligation and duly assert against an attaching obligee any event that results in extinguishing the obligation such as payment to the assignor Notwithstanding the provisions of the preceding paragraph
Article 466
if its nature does not permit the assignment this does not apply may be assigned
Even if a party to a claim mani

 99%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌ | 108/109 [00:39<00:00,  2.87it/s]

R05-36-E :  Y  -  N
Document:Article 705
if the person knew, at the time, that the obligation did not exist. may not demand the return of the money paid or thing delivered A person that has paid money or delivered anything as performance of an obligation
Question: A person that has paid money or delivered anything as performance of an obligation may not demand the return of the money paid or thing delivered if the person knew, or did not know due to gross negligence, that the obligation did not exist at the time the payment or the delivery was made.? No or Yes


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 109/109 [00:39<00:00,  2.75it/s]


In [47]:
print(count/len(data))

0.7339449541284404


In [20]:
def predict(model, tokenizer, files=["riteval_R01_en","riteval_R02_en","riteval_R03_en","riteval_R04_en"], output="../output/accuracy2/newpromt_"):
    for file in files:
        test_file = path_file+file+".xml"
        data = load_samples(test_file)
        
        acc = {}
        for template_prompt in list_prompt:
            template_prompt = template_prompt["prompt"]
            print(template_prompt)
            result = []
            count = 0
            for item in tqdm(data):
                label = item["label"]
                hypothesis = item["content"]
                premise = item["result"]
                #Important: You must use dot-product, not cosine_similarity
                text = prompting(premise, hypothesis, template_prompt)
                inputs = tokenizer(text, return_tensors="pt")["input_ids"].cuda()
                outputs = model.generate(inputs, max_new_tokens=10)
                output_text = format_output(tokenizer.decode(outputs[0]).replace(text, "").split("\n")[-1])
                if count < 1:
                    print(text)
                # if count < 3:
                #     print(output_text)
                if "yes" in output_text or "true" in output_text:
                    output_text = "Y"
                else:
                    output_text = "N"
                if output_text == label:
                    count+=1
                # print("predict label: ", output_text, "label: ", label)
            print("=======================================")
            print(template_prompt)
            print(count, "/", len(data))
            acc.update({template_prompt: count/len(data)})
        writefile(acc, output+file+".json")

In [8]:
predict(model, tokenizer, ["riteval_R01_en","riteval_R02_en","riteval_R03_en","riteval_R04_en"], "../output/accuracy2/newpromt_")

{{premise}}
Question: {{hypothesis}} True or False?


  2%|██▊                                                                                                                                                        | 2/111 [00:00<00:46,  2.36it/s]Token indices sequence length is longer than the specified maximum sequence length for this model (575 > 512). Running this sequence through the model will result in indexing errors


Article 5
(1) A minor must obtain the consent of the minor's legal representative to perform a juridical act;provided, however, that this does not apply to a juridical act for merely acquiring a right or being released from an obligation.
(2) A juridical act in contravention of the provisions of the preceding paragraph is voidable.
(3) Notwithstanding the provisions of paragraph (1), a minor may freely dispose of property that the legal representative has permitted the minor to dispose of for a specified purpose, to an extent that falls within the scope of that purpose. The same applies if the minor disposes of property that the legal representative has permitted the minor to dispose of without specifying a purpose.
Question: A contract of sales concluded by a minor may not be rescinded if it relates to daily life, even in cases the consent of the parental authority is not obtained. True or False?


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 111/111 [00:21<00:00,  5.25it/s]


{{premise}}
Question: {{hypothesis}} True or False?
75 / 111
{{premise}}
Question: {{hypothesis}} True or False? 


...

If {{premise}}, can we conclude that "{{hypothesis}}"? Yes or No 
81 / 101
Premise: "{{premise}}" If this premise is true, what does that tell us about whether it entails the hypothesis "{{hypothesis}}"?

Yes or No 


  1%|█▌                                                                                                                                                         | 1/101 [00:00<00:17,  5.85it/s]

Premise: "Article 5 (1) A minor must obtain the consent of the minor's legal representative to perform a juridical act;provided, however, that this does not apply to a juridical act for merely acquiring a right or being released from an obligation.
(2) A juridical act in contravention of the provisions of the preceding paragraph is voidable.
Article 6 (1) A minor who is permitted to conduct one or multiple types of business has the same capacity to act as an adult as far as that business is concerned." If this premise is true, what does that tell us about whether it entails the hypothesis "A juridical act by a minor who is permitted to conduct business may not be rescinded even if the judicial act does not concern the business."?

Yes or No 


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 101/101 [00:21<00:00,  4.67it/s]

Premise: "{{premise}}" If this premise is true, what does that tell us about whether it entails the hypothesis "{{hypothesis}}"?

Yes or No 
81 / 101


In [13]:
template_prompt = "Premise: {{premise}}\nHypothesis: {{hypothesis}}\nIs the hypothesis entailed by the premise? Yes or No (No explain)"
outpath = "../output/GPT_Llama2/prompt_6"
import os
if not os.path.exists(outpath):
	os.makedirs(outpath)
    
out_f = open(outpath+"/riteval_R05_en.txt", "w", encoding="utf-8")

for item in data:
    text = template_prompt.replace("{{premise}}", item["result"]).replace("{{hypothesis}}", item["content"])
    print(text)
    out_f.write(text+"\n")

Premise: Article 537(1) If one of the parties promises in a contract to render a certain performance to a third party, the third party has the right to claim that performance directly from the obligor.
(2) The validity of the contract referred to in the preceding paragraph is not impaired even if a third party does not exist or a third party is not specified at the time of its formation.
(3) In the case referred to in paragraph (1), rights of the third party accrue when the third party has manifested intention of availing of the benefit of the contract under that paragraph to the obligor.
Hypothesis: The validity of a third party beneficiary contract made by A with C, the mother of B, to give land X owned by A to B gratuitously, is not impaired even if B is an unborn child as of the time of the formation of the contract.
Is the hypothesis entailed by the premise? Yes or no (No explain)
Premise: Article 712 If a minor has inflicted damage on another person but did not have sufficient in

In [5]:
text = "EXAM\n\nAnswer by yes or no.\nDocument: Article 15 (1) The family court may decide to commence an assistance in respect of a person whose capacity to appreciate their own situation is inadequate due to a mental disorder, at the request of the person in question, that person's spouse, that person's relative within the fourth degree of kinship, the guardian, the guardian's supervisor, the curator, the curator's supervisor, or a public prosecutor; provided, however, that this does not apply to a person with respect to whom there are grounds as prescribed in Article 7 or the main clause of Article 11.\nArticle 11 The family court may decide to commence a curatorship in respect of a person whose capacity to appreciate their own situation is extremely inadequate due to a mental disorder, at the request of the person in question, the person's spouse, the person's relative within the fourth degree of kinship, the guardian, the guardian's supervisor, the assistant, the assistant's supervisor, or a public prosecutor; provided, however, that this does not apply to a person in respect of whom a cause set forth in Article 7 exists.\nQuestion: The family court may decide to commence an assistance also in respect of a person whose capacity to appreciate their own situation is extremely inadequate due to a mental disorder.?"

In [6]:
text

"EXAM\n\nAnswer by yes or no.\nDocument: Article 15 (1) The family court may decide to commence an assistance in respect of a person whose capacity to appreciate their own situation is inadequate due to a mental disorder, at the request of the person in question, that person's spouse, that person's relative within the fourth degree of kinship, the guardian, the guardian's supervisor, the curator, the curator's supervisor, or a public prosecutor; provided, however, that this does not apply to a person with respect to whom there are grounds as prescribed in Article 7 or the main clause of Article 11.\nArticle 11 The family court may decide to commence a curatorship in respect of a person whose capacity to appreciate their own situation is extremely inadequate due to a mental disorder, at the request of the person in question, the person's spouse, the person's relative within the fourth degree of kinship, the guardian, the guardian's supervisor, the assistant, the assistant's supervisor, 

In [7]:
inputs = tokenizer(text, return_tensors="pt")["input_ids"].cuda()
outputs = model.generate(inputs, max_new_tokens=10)
output_text = format_output(tokenizer.decode(outputs[0]).replace(text, "").split("\n")[-1])
output_text

'yes'

In [8]:
text = "If-Then Statements:\
Article 15(1): If a person's capacity to appreciate their own situation is inadequate due to a mental disorder, and if the person, their spouse, a relative up to the fourth degree of kinship, the guardian, the guardian's supervisor, the curator, the curator's supervisor, or a public prosecutor makes a request, then the family court may decide to commence assistance for that person.\
Article 11: If a person's capacity to appreciate their own situation is extremely inadequate due to a mental disorder, and if the person, their spouse, a relative up to the fourth degree of kinship, the guardian, the guardian's supervisor, the assistant, the assistant's supervisor, or a public prosecutor makes a request, then the family court may decide to commence curatorship for that person.\
Exception:\
Article 15(1): This rule does not apply to a person with respect to whom there are grounds as prescribed in Article 7 or the main clause of Article 11.\
Article 11: This rule does not apply to a person in respect of whom a cause set forth in Article 7 exists.\
Question: The family court may decide to commence an assistance also in respect of a person whose capacity to appreciate their own situation is extremely inadequate due to a mental disorder.?"

In [9]:
inputs = tokenizer(text, return_tensors="pt")["input_ids"].cuda()
outputs = model.generate(inputs, max_new_tokens=10)
output_text = format_output(tokenizer.decode(outputs[0]).replace(text, "").split("\n")[-1])
output_text

'no'

In [8]:
f1 = open("../output/final/final.submission/R05.task4.CAPTAIN2_riteval_R05_en.xml", "r")
f2 = open("../data/COLIEE2024statute_data-English/answer-task4", "r")

count = 0
for l1, l2 in zip(f1, f2):
    l1 = l1.strip().split(" ")[1]
    l2 = l2.strip().split("\t")[1]
    if l1 == l2: count += 1
    

In [9]:
count

90

In [10]:
count/109

0.8256880733944955